This code generates the model SNe for all of the models. Check up on Model 4, I think the mixed map has a normalization error. (map built in 3.4cModelVisualization)

In [1]:
#import packages and initialize settings
import time
import os
import numpy as np
from astropy.io import fits, ascii
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table


plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

import sys
# Work Laptop:     
#sys.path.append('/Users/mayker.1/Desktop/NessPythonFunctions')
# Work Desktop:    
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')
# Personal Laptop: sys.path.append('/Users/nessmayker/Desktop/NessPythonFunctions')
from FindNearestMC import nearestMCMethod
from SNeGenerationModels import runModels

In [2]:
#Get Galaxy names, distances, and fits file locations

# Unpack master table to get SNe information
database = Table.read('../Data/3.SNe+GalData.csv', format = "csv")
centerCoords = [(database["orient_ra"][i], database["orient_dec"][i]) for i in range(len(database))]   

database[0:1]



galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile,PACovFile,IFRFile,MixedMaps,IFRResizedMaps,MassMaps
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str80,str112,str97,str37,str54,str35
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/ESO097-013_alphaCO21_scaling.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_coverage2d.fits,/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/eso097-013_w1_gauss7p5.fits,../Data/MixedMaps/circinus_Mixed.fits,../Data/ReprojectedInfraredMaps/circinus_reshaped.fits,../Data/MassMaps/circinus_Mass.fits


In [3]:

#Make lists unique for galaxies (no repeating galaxies or other data)    
def unique(origGalsList, origDistList, imageFileList, errFileList, centerCoords, posAngList, inclList, covFileList, ifrFileList, massFileList, mixedFileList): 
    gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, massFiles, mixedFiles = [],[],[],[],[],[],[],[],[],[],[]
    for i in range(len(origGalsList)): 
        if origGalsList[i] not in gals: 
            if origGalsList[i] != "ngc1068" and origGalsList[i] != "ngc1672" and origGalsList[i] != "ngc4579":
                gals.append(origGalsList[i]) 
                dists.append(origDistList[i])
                images.append(imageFileList[i])
                errFiles.append(errFileList[i])
                coords.append(centerCoords[i])
                posAngs.append(posAngList[i])
                if inclList[i] == 90.0:
                    inclList[i] = 80.0
                inclinations.append(inclList[i]) 
                covFiles.append(covFileList[i])
                ifrFiles.append(ifrFileList[i])
                massFiles.append(massFileList[i])
                mixedFiles.append(mixedFileList[i])
            else:
                pass
    return(gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, massFiles, mixedFiles)

gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, massFiles, mixedFiles = unique(database["galaxy"], database["dist"],database["ImageFile150"], database["ErrFile150"], centerCoords, database["orient_posang"], database["orient_incl"], database["PACovFile"], database["IFRResizedMaps"], database["MassMaps"], database["MixedMaps"])
print(gals)


['circinus', 'ngc0253', 'ngc0628', 'ngc1087', 'ngc1097', 'ngc1365', 'ngc1433', 'ngc1511', 'ngc1559', 'ngc1566', 'ngc1637', 'ngc2775', 'ngc2997', 'ngc3239', 'ngc3627', 'ngc4254', 'ngc4303', 'ngc4321', 'ngc4424', 'ngc4457', 'ngc4496a', 'ngc4536', 'ngc4945', 'ngc5128', 'ngc5236', 'ngc5530', 'ngc6744', 'ngc7793']


In [5]:
    
# run models 1-4 and store results
galCol, counter, n55mod1, n65mod1, n55mod2, n65mod2, n55mod3, n65mod3 = [],[],[],[],[],[],[],[]

for i in range(len(gals)):
    
    ifr_hdu = pyfits.open(ifrFiles[i])
    ifr_map = ifr_hdu[0].data
    cov_hdu = pyfits.open(covFiles[i])
    cov_map = cov_hdu[0].data

    wcs     = WCS(cov_hdu[0].header, naxis=2)
    naxis   = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid    = np.indices((naxis[1],naxis[0]))
    ra, dec = wcs.wcs_pix2world(grid[1],grid[0],0) 

    f_cov = cov_map.flatten()
    f_ifr = ifr_map.flatten() 
    f_ra  = ra.flatten()
    f_dec = dec.flatten()

    keep = np.where((f_cov > 0.9) & np.isfinite(f_ifr))
    ifr  = f_ifr[keep]
    star_ra   = f_ra[keep]
    star_dec  = f_dec[keep]
    
    
    raArr1, decArr1, dxArr1, dyArr1 = runModels(gals[i], images[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=1)
    raArr2, decArr2, dxArr2, dyArr2 = runModels(gals[i], images[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=2)
    raArr3, decArr3, dxArr3, dyArr3 = runModels(gals[i], images[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=3, starLight = ifr, starRa = star_ra, starDec = star_dec)
    #raArr4, decArr4, dxArr4, dyArr4 = runModels(gals[i], mixedFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=2)

    n55_mod1, n65_mod1 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr1, decArr1)
    n55_mod2, n65_mod2 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr2, decArr2)
    n55_mod3, n65_mod3 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr3, decArr3)
    #n55_mod4, n65_mod4 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr4, decArr4)
    
    if (type(n55_mod1) != float and type(n55_mod1) != np.float64):
        for j in range(len(n55_mod1)):
            galCol.append(gals[i])
            counter.append(j)
            n55mod1.append(n55_mod1[j])
            n55mod2.append(n55_mod2[j])
            n55mod3.append(n55_mod3[j])
            #n55mod4.append(n55_mod4[j])
    else:
        for j in range(100):
            galCol.append(gals[i])
            counter.append(j)
            n55mod1.append(np.float('nan'))
            n55mod2.append(np.float('nan'))
            n55mod3.append(np.float('nan'))
            #n55mod4.append(np.float('nan'))           
        
    if (type(n65_mod1) != float and type(n65_mod1) != np.float64):
        for j in range(len(n65_mod1)):
            n65mod1.append(n65_mod1[j])
            n65mod2.append(n65_mod2[j])
            n65mod3.append(n65_mod3[j])
            #n65mod4.append(n65_mod4[j])
    else:
        for j in range(100):
            n65mod1.append(np.float('nan'))
            n65mod2.append(np.float('nan'))
            n65mod3.append(np.float('nan'))
            #n65mod4.append(np.float('nan'))      
    
results = Table({"Galaxy": galCol,
                "ModelNum": counter,
                "n55Mod1": n55mod1,
                "n65Mod1": n65mod1,
                "n55Mod2": n55mod2,
                "n65Mod2": n65mod2,
                "n55Mod3": n55mod3,
                "n65Mod3": n65mod3})

results.write("../Data/AllModels.csv", format = "csv", overwrite = True)

        
        
    

at Mass cutoff A for circinus
Pixels with Mass > 10**5.5 5804
Nearest 55 [   0.         1505.20277507  509.10130742    0.          318.42908525
  180.5462145    60.20266317  441.65686734  389.6638969   124.44318799
 1089.95758892  319.92162242  249.21126297  322.63651798  361.23398699
  425.57560929 1029.37639085   97.08276367 1171.48538724  129.82887832
   60.06828079  242.52151837  781.29419179    0.          843.59636429
  291.16724731  180.5462145    60.14643763   97.04240876    0.
   97.00514421 1340.10196473  300.17860698  129.91599396  620.13466802
  387.90492429   60.06701698  338.07708886  240.46266784  147.70902411
   60.0988245   601.74936155    0.          537.71151631 1712.49597644
    0.          147.74839436  147.71966019  215.3324768   775.55116599
    0.          300.36494796   96.97488608  318.48478646    0.
  303.25356139  316.10735855 2297.78132413  129.93611028  301.13699882
   64.97807865  426.51912782    0.            0.           65.036008
    0.            0.  

at Mass cutoff A for ngc0253
Pixels with Mass > 10**5.5 36463
Nearest 55 [  98.21976402  352.59304988    0.            0.          638.62562044
  399.93943173   89.17957859  264.65488178  275.53955747    0.
    0.          357.37613084  924.64500624  729.92750996    0.
   56.32078743 2638.11211303  615.12717657  133.09805671    0.
    0.            0.            0.            0.          415.950509
    0.           98.51013358  230.32423602    0.          485.10024951
    0.         1592.85578917  354.87101055   98.27239101  537.09102397
    0.         1007.54018167    0.          227.45399338    0.
    0.          133.17150893  415.38680626    0.            0.
  615.99913268 1104.91041949    0.          648.81294212   56.38514219
    0.          275.29603875   88.91167196  196.52139267    0.
    0.          564.38643304  338.60828313  230.39375048    0.
    0.            0.            0.         1836.67433957    0.
    0.           98.44808555  688.2923639     0.            0.
 1243.0

at Mass cutoff A for ngc0628
Pixels with Mass > 10**5.5 454353
Nearest 55 [   0.          162.93883102  381.45025362  546.16287011  163.94668456
    0.          185.73219371    0.          417.38169858   39.34547182
  258.00505578    0.          191.18627226  313.61820657   42.91061834
  338.96011806    9.64350496  137.41884969    0.            0.
  216.32936832  193.27318253   34.53966449    0.          406.44646914
  426.42466042    0.          802.93160134   30.31812443 1277.68630937
    0.          410.89627186    9.5557429   309.17882158  163.43466808
  511.88810914    0.            0.          277.1895469     0.
    0.            9.55574762    0.           75.20540883    0.
   57.3344105     0.          238.69081696    9.55574631    0.
  238.89334424  343.52540823  250.83808432  339.11104369    0.
  281.45749228    0.         1686.27161374  177.53020324    0.
  245.85535163    0.          192.03811302  293.60695921  105.11315339
  106.59199238  129.58765091  129.55578801    0.   

at Mass cutoff A for ngc1087
Pixels with Mass > 10**5.5 43276
Nearest 55 [2173.38941788    0.          349.42240951  284.34129638  488.68884322
 3009.71655625 3404.2498671   224.33039473  208.03139196  494.32335898
  555.84106872 2577.37825864 3710.09901343    0.          511.72723014
 2258.36891057   76.27200665 2445.87963352  222.26461238 1284.12982476
   23.05534713   46.11069426  706.03151688  571.86341479  351.92789906
    0.         1248.69244141  313.55407117 2048.93960281    0.
 2849.36805224   66.81047511   75.70290122  989.47476574  607.97686149
 3081.2343496  1164.31087179 1855.74829931 2614.12616567 1607.85486796
  630.36450118  161.38743019   93.97517449 2447.14206664 1633.02455012
  169.8353555   733.52317202  380.87285491 1321.41955562 1852.78694572
 2990.93685989  299.71952141 1841.19396021  220.23713293  518.31426339
  203.68110772  249.59902725  959.53175788 2502.85966613    0.
 1380.1440155   119.7957876  2312.35421658 1945.36522542  324.64113164
  222.19160051 4191.

at Mass cutoff A for ngc1097
Pixels with Mass > 10**5.5 42167
Nearest 55 [1302.45748321   38.52239752   45.58076916    0.          679.30891976
  307.42073885   74.09778147    0.          786.42020146 1148.50420336
    0.          110.98087724 1828.58671586  488.77382684   47.95388078
  323.7011704   255.81196561    0.            0.          102.63388261
    0.          115.57189169  792.88851995   85.27404244    0.
 1242.40203585 1229.94529368 2293.23456511  410.57223      85.26730458
  389.02416523   74.1022472   723.67581728  401.87534331  397.97416714
    0.          130.63221601  251.17970195  314.12359407 1127.76717569
  136.73596356  945.78165804  143.86628919 1019.01426713 1018.18733314
    0.          199.41900209   69.43743472    0.           85.26835552
  138.92044132    0.          377.54654918    0.          251.12117045
    0.          289.14733352 2042.02706823  115.56059308   69.43487464
    0.           95.91034808  467.54369539  978.31387938  887.86961826
  170.540648

...

at Mass cutoff A for ngc5128
Pixels with Mass > 10**5.5 11514
Nearest 55 [   0.          207.3979955     0.         2589.09603322  195.8902008
    0.          899.90306898  175.49317567    0.         1750.32969708
 3047.49525812 1120.71394834  229.40222184 1368.4074765   383.00873092
  556.379454     40.63196563   47.10762116 1017.58027189  392.58738396
   80.40955232  116.36148679  354.21258671 1007.23688303  579.93200646
   40.64618198    0.         1380.08916206  452.60560726    0.
   71.08695605  246.1819302   351.17547626  695.19240105  103.75764206
    0.           47.11255445    0.            0.          887.29938794
 1303.78512757  951.46190751  574.51123123    0.         1183.71652316
    0.          226.98944205   47.09671072  512.41450508 1731.82610767
 1791.75779961  788.99602205    0.          391.61124535  222.28722735
  579.63656964  660.96782747  950.8575075   348.58038327  562.82958398
  383.08381448 1189.55230791 1258.36647858   90.34473628 2090.58359759
  211.4815619

at Mass cutoff A for ngc5236
Pixels with Mass > 10**5.5 468368
Nearest 55 [  18.34983934   18.34540955    0.          329.36490262  318.10014304
    0.          160.8475685    53.53626377  226.69221379   24.85409382
    0.          121.04317604  220.88316535  347.93665898    0.
   37.28096869    0.            0.            0.            0.
  240.88255363   80.29472327    0.           24.85473717    0.
    0.            0.           12.42671481    0.           12.42470469
    0.           57.54320195   62.23921443    0.            0.
    0.          192.93414176  241.47955321   46.6300169     0.
    0.           12.42734759    0.           16.76640872  577.67798879
    0.          137.97774256    0.          225.13672294  267.5158171
    0.            0.          198.0242948     0.            0.
    0.           86.75494519    0.          270.79792048    0.
    0.          107.05198178    0.          176.43297791    0.
  375.09416319    0.            0.          413.00611055  128.910535

at Mass cutoff A for ngc5530
Pixels with Mass > 10**5.5 36885
Nearest 55 [1652.86270582   77.40358057 2648.94742536   26.33673216  310.20640406
  228.87595245    0.         1345.7991841   282.20161224  576.68577157
  114.92247919    0.         2701.72945506  992.55584712 1647.49518666
  589.2636038     0.            0.          333.60551221 1334.94522171
  105.14912974 2308.51188476  128.52166492  410.94067728 1079.47194408
   77.38671926  512.47113207  361.22208553 1139.92567463 1310.7042998
  461.48273513 1230.67446939  410.7996103   154.49423085  102.72917737
  318.96275026    0.            0.         1191.79825352  282.61006359
  124.56412588 3040.32520235   52.88866823  105.13058337    0.
  128.88962451  523.67282623 1883.68701295  432.6428638   128.50790003
 2626.66141894  712.94674848  373.68492773 1236.93547512 1463.8501581
  187.48816087  544.96990926  313.23605947 2795.45689533 2763.13343242
  623.2432229     0.          509.10316216 1070.08810342    0.
  816.25466727 1688.00

at Mass cutoff A for ngc6744
Pixels with Mass > 10**5.5 147902
Nearest 55 [ 227.65612792   42.98504275    0.            0.            0.
  100.30095482  210.7265593   408.67428514   85.95422531  167.04397405
  112.61688734   85.23367389  107.26944869  328.49227758  642.46211525
  111.48313179  269.34659935   23.32613971  278.31020542   22.13915698
    0.          758.8874238     0.          282.01011791  999.37319374
  182.24988058  100.33891129  222.97475314  106.29327409  122.96610533
  476.88967801 1054.26247656  282.07626335    0.          214.52407739
    0.           31.73436575  206.97023649  172.58818872  293.46816398
  446.18865867   14.3247426   225.22122258   22.13708149    0.
  107.79016712  169.57145203    0.          378.25409711   78.23767304
    0.           28.66036331  120.80593023  257.94094237   71.17980335
  223.23904977  169.69698074  445.96909889  341.88409271  176.67315635
  445.41515482  215.15862399  453.71201877   85.29258398  116.8170828
  132.7083077     0.

at Mass cutoff A for ngc7793
Pixels with Mass > 10**5.5 1611
Nearest 55 [ 756.39608395    0.         2041.03850639 1549.29245112  873.02395216
 1153.16683231  761.94814517  311.34078814  379.2790405  1000.57403473
  121.63180243  317.01077586 1746.06714606    0.          593.49734314
  690.83729178  962.38806091  316.90599411   73.06535277  355.11236767
  599.94870272 1537.59936697 1669.06283734  821.66190971 1820.01251203
 1392.77189593 1951.78017583  661.15743766 1806.47634474  929.18306785
 1804.30569805    0.          341.00879037  292.25241775 1082.14270296
  154.89469008 1650.13682399  606.95837741 1625.70074321 1402.25332523
 2965.80463685  738.22122521  327.45939823  202.9211448   556.90180035
 1341.68738214  661.46042106    0.          303.71048732  556.35490928
  333.15020706 1836.40225206    0.          492.91991047   73.07183083
  105.38840073 1739.43802921  764.19147676   37.9152069   154.90388976
  813.69693165 1588.87821435   75.82885022  607.00186265  693.66658334
 1356